In [23]:
import numpy as np # linear algebra
import pandas as pd
import sqlite3

df_olist_customers = pd.read_csv('/Users/annanefedova/Desktop/Учеба /Developers Institute/repozitories/DI-Bootcamp/week7/day2/Brazilian E-Commerce Dive Deep using SQL/Brazilian E-Commerce Dive Deep using SQL/olist_products_dataset.csv/olist_customers_dataset.csv')
df_olist_sellers = pd.read_csv('/Users/annanefedova/Desktop/Учеба /Developers Institute/repozitories/DI-Bootcamp/week7/day2/Brazilian E-Commerce Dive Deep using SQL/Brazilian E-Commerce Dive Deep using SQL/olist_products_dataset.csv/olist_sellers_dataset.csv')
df_olist_order_reviews= pd.read_csv('/Users/annanefedova/Desktop/Учеба /Developers Institute/repozitories/DI-Bootcamp/week7/day2/Brazilian E-Commerce Dive Deep using SQL/Brazilian E-Commerce Dive Deep using SQL/olist_products_dataset.csv/olist_order_reviews_dataset.csv')
df_olist_order_items= pd.read_csv('/Users/annanefedova/Desktop/Учеба /Developers Institute/repozitories/DI-Bootcamp/week7/day2/Brazilian E-Commerce Dive Deep using SQL/Brazilian E-Commerce Dive Deep using SQL/olist_products_dataset.csv/olist_order_items_dataset.csv')
df_olist_products= pd.read_csv('/Users/annanefedova/Desktop/Учеба /Developers Institute/repozitories/DI-Bootcamp/week7/day2/Brazilian E-Commerce Dive Deep using SQL/Brazilian E-Commerce Dive Deep using SQL/olist_products_dataset.csv/olist_products_dataset.csv')
df_olist_geolocation= pd.read_csv('/Users/annanefedova/Desktop/Учеба /Developers Institute/repozitories/DI-Bootcamp/week7/day2/Brazilian E-Commerce Dive Deep using SQL/Brazilian E-Commerce Dive Deep using SQL/olist_products_dataset.csv/olist_geolocation_dataset.csv')
df_product_category_name_translation= pd.read_csv('/Users/annanefedova/Desktop/Учеба /Developers Institute/repozitories/DI-Bootcamp/week7/day2/Brazilian E-Commerce Dive Deep using SQL/Brazilian E-Commerce Dive Deep using SQL/olist_products_dataset.csv/product_category_name_translation.csv')
df_olist_orders = pd.read_csv('/Users/annanefedova/Desktop/Учеба /Developers Institute/repozitories/DI-Bootcamp/week7/day2/Brazilian E-Commerce Dive Deep using SQL/Brazilian E-Commerce Dive Deep using SQL/olist_products_dataset.csv/olist_orders_dataset.csv')
df_olist_order_payments= pd.read_csv('/Users/annanefedova/Desktop/Учеба /Developers Institute/repozitories/DI-Bootcamp/week7/day2/Brazilian E-Commerce Dive Deep using SQL/Brazilian E-Commerce Dive Deep using SQL/olist_products_dataset.csv/olist_order_payments_dataset.csv')

df_olist_customers.head()

from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

# export the dataframe as a table 'playstore' to the sqlite engine
df_olist_customers.to_sql("olist_customers", con =engine)
df_olist_sellers.to_sql("olist_sellers", con =engine)
df_olist_order_reviews.to_sql("olist_order_reviews", con =engine)
df_olist_order_items.to_sql("olist_order_items", con =engine)
df_olist_products.to_sql("olist_products_dataset", con =engine)
df_olist_geolocation.to_sql("olist_geolocation", con =engine)
df_product_category_name_translation.to_sql("product_category_name_translation", con =engine)
df_olist_orders.to_sql("olist_orders", con =engine)
df_olist_order_payments.to_sql("olist_order_payments", con =engine)
df_olist_order_payments.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [28]:
sql='''

SELECT * FROM olist_sellers WHERE seller_id IS NULL;

'''


df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,index,seller_id,seller_zip_code_prefix,seller_city,seller_state


In [29]:
sql='''

SELECT * FROM olist_customers WHERE customer_id IS NULL


'''


df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,index,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state


3. Query 1: Count and Percentage of Orders Purchased in Jan 2018 with 5 Review Score

Write and execute a SQL query to count the number of orders purchased in January 2018 that have a review score of 5 and calculate the percentage of such orders.

In [26]:
sql = """
WITH jan_orders AS (
  SELECT o.order_id
  FROM olist_orders AS o
  WHERE DATE(o.order_purchase_timestamp) >= DATE('2018-01-01')
    AND DATE(o.order_purchase_timestamp) <  DATE('2018-02-01')
),
last_review AS (
  SELECT r.order_id, r.review_score
  FROM (
    SELECT order_id, MAX(review_creation_date) AS max_dt
    FROM olist_order_reviews
    GROUP BY order_id
  ) AS x
  JOIN olist_order_reviews r
    ON r.order_id = x.order_id
   AND r.review_creation_date = x.max_dt
)
SELECT
  COUNT(*) AS total_jan_orders,
  SUM(CASE WHEN lr.review_score = 5 THEN 1 ELSE 0 END) AS five_star_orders,
  ROUND(
    100.0 * SUM(CASE WHEN lr.review_score = 5 THEN 1 ELSE 0 END) / NULLIF(COUNT(*),0),
    2
  ) AS pct_5star
FROM jan_orders jo
LEFT JOIN last_review lr ON lr.order_id = jo.order_id;
"""
pd.read_sql_query(sql, con=engine)

,total_jan_orders,five_star_orders,pct_5star
0,7280,4080,56.04


4. Query 2: Customer Purchase Trend Year-on-Year

Write and execute a SQL query to analyze the customer purchase trend year-on-year.

In [27]:
sql = """
WITH yearly AS (
  SELECT
    CAST(STRFTIME('%Y', order_purchase_timestamp) AS INT) AS yr,
    COUNT(*) AS orders_cnt,
    COUNT(DISTINCT customer_id) AS customers_cnt
  FROM olist_orders
  GROUP BY CAST(STRFTIME('%Y', order_purchase_timestamp) AS INT)
)
SELECT
  yr,
  orders_cnt,
  customers_cnt,
  ROUND(1.0 * orders_cnt / NULLIF(customers_cnt, 0), 3) AS orders_per_customer,
  LAG(orders_cnt) OVER (ORDER BY yr) AS prev_orders_cnt,
  ROUND(
    100.0 * (orders_cnt - LAG(orders_cnt) OVER (ORDER BY yr))
    / NULLIF(LAG(orders_cnt) OVER (ORDER BY yr), 0),
    2
  ) AS yoy_orders_pct,
  LAG(customers_cnt) OVER (ORDER BY yr) AS prev_customers_cnt,
  ROUND(
    100.0 * (customers_cnt - LAG(customers_cnt) OVER (ORDER BY yr))
    / NULLIF(LAG(customers_cnt) OVER (ORDER BY yr), 0),
    2
  ) AS yoy_customers_pct
FROM yearly
ORDER BY yr;

"""
pd.read_sql_query(sql, con=engine)

,yr,orders_cnt,customers_cnt,orders_per_customer,prev_orders_cnt,yoy_orders_pct,prev_customers_cnt,yoy_customers_pct
0,2016,329,329,1.0,NaN,NaN,NaN,NaN
1,2017,45101,45101,1.0,329.0,13608.51,329.0,13608.51
2,2018,54011,54011,1.0,45101.0,19.76,45101.0,19.76


5. Query 3: Average Order Values of Customers

Write and execute a SQL query to calculate the average order values of customers.

In [30]:
sql = """
WITH order_value AS (
  SELECT
    p.order_id,
    SUM(p.payment_value) AS order_value
  FROM olist_order_payments AS p
  GROUP BY p.order_id
),
cust_orders AS (
  SELECT
    o.customer_id,
    ov.order_value
  FROM olist_orders AS o
  JOIN order_value AS ov
    ON ov.order_id = o.order_id
)
SELECT
  customer_id,
  COUNT(*)                      AS orders_cnt,
  ROUND(SUM(order_value), 2)    AS revenue_sum,
  ROUND(AVG(order_value), 2)    AS avg_order_value -- AOV клиента
FROM cust_orders
GROUP BY customer_id
ORDER BY avg_order_value DESC
"""
pd.read_sql_query(sql, con=engine)

,customer_id,orders_cnt,revenue_sum,avg_order_value
0,1617b1357756262bfa56ab541c47bc16,1,13664.08,13664.08
1,ec5b2ba62e574342386871631fafd3fc,1,7274.88,7274.88
2,c6e2731c5b391845f6800c97401a43a9,1,6929.31,6929.31
3,f48d464a0baaea338cb25f816991ab1f,1,6922.21,6922.21
4,3fd6777bbce08a352fddd04e4a7cc8f6,1,6726.66,6726.66
...,...,...,...,...
99435,184e8e8e48937145eb96c721ef1f0747,1,10.07,10.07
99436,a790343ca6f3fee08112d678b43aa7c5,1,9.59,9.59
99437,a73c1f73f5772cf801434bf984b0b1a7,1,0.00,0.00
99438,3532ba38a3fd242259a514ac2b6ae6b6,1,0.00,0.00


6. Query 4: Top 5 Cities with Highest Revenue from 2016 to 2018

Write and execute a SQL query to find the top 5 cities with the highest revenue from 2016 to 2018.

In [31]:
sql = """
WITH order_rev AS (
  SELECT
    order_id,
    SUM(payment_value) AS rev
  FROM olist_order_payments
  GROUP BY order_id
),
orders_2016_2018 AS (
  SELECT
    order_id,
    customer_id
  FROM olist_orders
  WHERE DATE(order_purchase_timestamp) >= DATE('2016-01-01')
    AND DATE(order_purchase_timestamp) <  DATE('2019-01-01')
)
SELECT
  c.customer_city,
  ROUND(SUM(orv.rev), 2) AS city_revenue
FROM orders_2016_2018 o
JOIN olist_customers c
  ON c.customer_id = o.customer_id
JOIN order_rev orv
  ON orv.order_id = o.order_id
GROUP BY c.customer_city
ORDER BY city_revenue DESC
LIMIT 5
"""
pd.read_sql_query(sql, con=engine)

,customer_city,city_revenue
0,sao paulo,2203373.09
1,rio de janeiro,1161927.36
2,belo horizonte,421765.12
3,brasilia,354216.78
4,curitiba,247392.48


7. Query 5: State Wise Revenue Table Between 2016 to 2018

Write and execute a SQL query to create a state-wise revenue table between 2016 to 2018.

In [32]:
sql = """
WITH order_rev AS (
  SELECT
    order_id,
    SUM(payment_value) AS rev
  FROM olist_order_payments
  GROUP BY order_id
),
orders_yr AS (
  SELECT
    o.order_id,
    o.customer_id,
    CAST(STRFTIME('%Y', o.order_purchase_timestamp) AS INT) AS yr
  FROM olist_orders o
  WHERE DATE(o.order_purchase_timestamp) >= DATE('2016-01-01')
    AND DATE(o.order_purchase_timestamp) <  DATE('2019-01-01')
)
SELECT
  c.customer_state AS state,
  ROUND(SUM(CASE WHEN oy.yr = 2016 THEN orv.rev ELSE 0 END), 2) AS rev_2016,
  ROUND(SUM(CASE WHEN oy.yr = 2017 THEN orv.rev ELSE 0 END), 2) AS rev_2017,
  ROUND(SUM(CASE WHEN oy.yr = 2018 THEN orv.rev ELSE 0 END), 2) AS rev_2018,
  ROUND(SUM(orv.rev), 2) AS total_2016_2018
FROM orders_yr oy
JOIN olist_customers c ON c.customer_id = oy.customer_id
JOIN order_rev orv      ON orv.order_id   = oy.order_id
GROUP BY c.customer_state
ORDER BY total_2016_2018 DESC;

"""
pd.read_sql_query(sql, con=engine)

,state,rev_2016,rev_2017,rev_2018,total_2016_2018
0,SP,16885.54,2561862.91,3419478.51,5998226.96
1,RJ,13407.58,1055954.48,1075017.63,2144379.69
2,MG,5642.97,854891.64,1011722.65,1872257.26
3,RS,4790.70,424693.30,461414.54,890898.54
4,PR,2599.97,353782.25,454774.16,811156.38
5,SC,2730.42,277452.28,342903.73,623086.43
6,BA,995.34,284324.38,331326.10,616645.82
7,DF,1200.11,157089.83,196851.14,355141.08
8,GO,1223.06,164005.35,184863.90,350092.31
9,ES,1067.14,143116.93,181783.48,325967.55


8. Query 6: Top Successful Sellers in Terms of Goods Sold, Revenue, and Customer Count

Write and execute a SQL query to identify the top successful sellers in terms of the number of goods sold, total revenue, customer count, and sellers with the highest 5-star ratings.

In [34]:
sql = """
WITH item_agg AS (
  SELECT
    oi.seller_id,
    COUNT(*)                                  AS items_sold,
    SUM(oi.price)                             AS revenue_items,
    SUM(oi.price + oi.freight_value)          AS gmv_incl_freight
  FROM olist_order_items AS oi
  GROUP BY oi.seller_id
),
seller_customers AS (
  SELECT
    oi.seller_id,
    COUNT(DISTINCT o.customer_id) AS unique_customers
  FROM olist_order_items AS oi
  JOIN olist_orders AS o
    ON o.order_id = oi.order_id
  GROUP BY oi.seller_id
),
seller_reviews AS (
  SELECT
    oi.seller_id,
    COUNT(DISTINCT r.order_id) AS total_reviewed_orders,
    COUNT(DISTINCT CASE WHEN r.review_score = 5 THEN r.order_id END) AS five_star_orders
  FROM olist_order_items AS oi
  JOIN olist_order_reviews AS r
    ON r.order_id = oi.order_id
  GROUP BY oi.seller_id
),
base AS (
  SELECT
    ia.seller_id,
    ia.items_sold,
    ROUND(ia.revenue_items, 2)      AS revenue_items,
    ROUND(ia.gmv_incl_freight, 2)   AS gmv_incl_freight,
    sc.unique_customers,
    COALESCE(sr.total_reviewed_orders, 0) AS total_reviewed_orders,
    COALESCE(sr.five_star_orders, 0)      AS five_star_orders,
    ROUND(
      1.0 * COALESCE(sr.five_star_orders, 0) / NULLIF(COALESCE(sr.total_reviewed_orders, 0), 0),
      4
    ) AS five_star_rate
  FROM item_agg ia
  LEFT JOIN seller_customers sc ON sc.seller_id = ia.seller_id
  LEFT JOIN seller_reviews  sr ON sr.seller_id = ia.seller_id
),
ranked AS (
  SELECT
    b.*,
    RANK() OVER (ORDER BY b.items_sold       DESC) AS r_items,
    RANK() OVER (ORDER BY b.revenue_items    DESC) AS r_revenue,
    RANK() OVER (ORDER BY b.unique_customers DESC) AS r_customers,
    RANK() OVER (ORDER BY b.five_star_rate   DESC) AS r_five_star_rate,
    -- простой комбинированный скор: чем меньше сумма рангов, тем лучше
    (RANK() OVER (ORDER BY b.items_sold       DESC)
   +  RANK() OVER (ORDER BY b.revenue_items    DESC)
   +  RANK() OVER (ORDER BY b.unique_customers DESC)
   +  RANK() OVER (ORDER BY b.five_star_rate   DESC)) AS composite_rank
  FROM base b
)
SELECT
  seller_id,
  items_sold,
  revenue_items,
  gmv_incl_freight,
  unique_customers,
  total_reviewed_orders,
  five_star_orders,
  five_star_rate,
  r_items, r_revenue, r_customers, r_five_star_rate,
  composite_rank
FROM ranked
ORDER BY composite_rank ASC, revenue_items DESC
LIMIT 10;  -- топ-10 интегрально успешных продавцов

"""
pd.read_sql_query(sql, con=engine)

,seller_id,items_sold,revenue_items,gmv_incl_freight,unique_customers,total_reviewed_orders,five_star_orders,five_star_rate,r_items,r_revenue,r_customers,r_five_star_rate,composite_rank
0,c3cfdc648177fdbbbb35635a37472c53,307,43048.18,49127.81,278,277,203,0.7329,61,52,58,877,1048
1,fa1c13f2614d7b5c4749cbc52fecda94,586,194042.03,204084.73,585,581,393,0.6764,21,4,19,1042,1086
2,620c87c171fb2a6dd6e8bb4dec959fc6,798,114774.50,128909.71,740,733,490,0.6685,15,13,14,1056,1098
3,fe2032dab1a61af8794248c8196565c9,306,65959.61,70680.95,294,290,203,0.7000,62,23,57,976,1118
4,fa40cc5b934574b62717c68f3d678b6d,340,16055.87,20586.77,307,305,224,0.7344,51,158,55,871,1135
5,9f505651f4a6abe901a56cdc21508025,430,26361.82,32391.86,397,394,270,0.6853,33,96,33,1024,1186
6,7299e27ed73d2ad986de7f7c77d919fa,361,34725.43,41581.08,338,338,229,0.6775,47,68,44,1039,1198
7,c70c1b0d8ca86052f45a432a38b73958,338,36537.37,40476.28,261,259,177,0.6834,55,65,62,1027,1209
8,0241d4d5d36f10f80c644447315af0bd,246,31397.70,38251.61,238,237,162,0.6835,75,81,70,1026,1252
9,edb1ef5e36e0c8cd84eb3c9b003e486d,175,79284.55,83189.65,166,166,115,0.6928,126,20,110,997,1253


9. Query 7: Delivery Success Rate Across States

Write and execute a SQL query to calculate the delivery success rate across different states.

In [37]:
sql = """
WITH base AS (
  SELECT
    o.order_id,
    o.order_status,
    c.customer_state
  FROM olist_orders AS o
  JOIN olist_customers AS c
    ON c.customer_id = o.customer_id
)
SELECT
  customer_state AS state,
  COUNT(*) AS total_orders,
  SUM(CASE WHEN order_status = 'delivered' THEN 1 ELSE 0 END) AS delivered_orders,
  ROUND(
    100.0 * SUM(CASE WHEN order_status = 'delivered' THEN 1 ELSE 0 END) / NULLIF(COUNT(*), 0),
    2
  ) AS success_rate_pct
FROM base
GROUP BY customer_state
ORDER BY success_rate_pct DESC, total_orders DESC;

"""
pd.read_sql_query(sql, con=engine)

,state,total_orders,delivered_orders,success_rate_pct
0,AC,81,80,98.77
1,AP,68,67,98.53
2,ES,2033,1995,98.13
3,MS,715,701,98.04
4,AM,148,145,97.97
5,TO,280,274,97.86
6,RS,5466,5345,97.79
7,RN,485,474,97.73
8,MT,907,886,97.68
9,MG,11635,11354,97.58


10. Query 8: Preferred Form of Payment for Different Categories

Write and execute a SQL query to find the preferred form of payment for different product categories.


In [43]:
sql = """
WITH order_pay AS (           -- выручка по заказу и типу оплаты
  SELECT
    p.order_id,
    p.payment_type,
    SUM(p.payment_value) AS pay_sum
  FROM olist_order_payments AS p
  GROUP BY p.order_id, p.payment_type
),
order_cat AS (                -- категория(и) заказа: order_id × category
  SELECT
    oi.order_id,
    COALESCE(t.product_category_name_english, pr.product_category_name) AS category_en
  FROM olist_order_items AS oi
  JOIN olist_products_dataset AS pr USING(product_id)             -- <— тут фикс
  LEFT JOIN product_category_name_translation AS t
         ON t.product_category_name = pr.product_category_name
  GROUP BY oi.order_id, category_en
),
cat_pay AS (                  -- выручка категории по типам оплаты
  SELECT
    oc.category_en,
    op.payment_type,
    SUM(op.pay_sum) AS revenue_by_type
  FROM order_cat AS oc
  JOIN order_pay AS op ON op.order_id = oc.order_id
  GROUP BY oc.category_en, op.payment_type
),
ranked AS (                   -- берём лучший тип оплаты по выручке
  SELECT
    category_en,
    payment_type,
    revenue_by_type,
    ROW_NUMBER() OVER (PARTITION BY category_en ORDER BY revenue_by_type DESC) AS rn,
    SUM(revenue_by_type) OVER (PARTITION BY category_en) AS category_total
  FROM cat_pay
)
SELECT
  category_en                           AS category,
  payment_type                          AS preferred_payment_type,
  ROUND(revenue_by_type, 2)             AS preferred_payment_revenue,
  ROUND(category_total, 2)              AS category_revenue_total,
  ROUND(100.0 * revenue_by_type / NULLIF(category_total, 0), 2) AS share_pct
FROM ranked
WHERE rn = 1
ORDER BY category;

"""
pd.read_sql_query(sql, con=engine)

,category,preferred_payment_type,preferred_payment_revenue,category_revenue_total,share_pct
0,None,credit_card,162989.15,214503.20,75.98
1,agro_industry_and_commerce,credit_card,54058.09,78379.19,68.97
2,air_conditioning,credit_card,47933.20,62045.33,77.26
3,art,credit_card,15420.10,28815.93,53.51
4,arts_and_craftmanship,credit_card,1281.19,2241.54,57.16
...,...,...,...,...,...
69,stationery,credit_card,225545.97,281098.93,80.24
70,tablets_printing_image,credit_card,7265.50,9217.91,78.82
71,telephony,credit_card,300983.97,398760.08,75.48
72,toys,credit_card,461801.00,566925.14,81.46


11. Query 9: Distance Between Cities

Write and execute a SQL query to calculate the distance between cities.

In [46]:
from sqlalchemy import text

ddl = """
CREATE TABLE IF NOT EXISTS city_centroids AS
SELECT
  geolocation_city  AS city,
  geolocation_state AS state,
  AVG(geolocation_lat) AS lat,
  AVG(geolocation_lng) AS lng
FROM olist_geolocation
GROUP BY geolocation_city, geolocation_state;
"""

# В SQLAlchemy 2.x правильно так:
with engine.begin() as conn:
    conn.exec_driver_sql(ddl)

# Проверка: теперь уже SELECT через pandas
pd.read_sql("SELECT * FROM city_centroids LIMIT 5;", con=engine)


,city,state,lat,lng
0,* cidade,PR,-25.571748,-49.334374
1,...arraial do cabo,RJ,-22.969370,-42.029834
2,4o. centenario,PR,-24.271860,-53.069433
3,4º centenario,PR,-24.279801,-53.074723
4,abadia de goias,GO,-16.767063,-49.438089


In [48]:
import math
import pandas as pd

def haversine_km(lat1, lon1, lat2, lon2):
    if None in (lat1, lon1, lat2, lon2):
        return None
    R = 6371.0088  # км
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlmb = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlmb/2)**2
    c = 2 * math.asin(math.sqrt(a))
    return R * c

# Регистрируем функцию в текущем sqlite-соединении
raw = engine.raw_connection()          # достаём нативный sqlite3 connection
raw.create_function("haversine_km", 4, haversine_km)


In [50]:
sql = """
SELECT
  a.city AS city_a, a.state AS state_a,
  b.city AS city_b, b.state AS state_b,
  ROUND(haversine_km(a.lat, a.lng, b.lat, b.lng), 2) AS distance_km
FROM city_centroids a
JOIN city_centroids b
  ON a.city = 'sao paulo' AND a.state = 'SP'
 AND b.city = 'rio de janeiro' AND b.state = 'RJ';
"""
pd.read_sql(sql, con=raw)


/var/folders/s1/j1lr__816ks_mg4_h_l5z27h0000gn/T/ipykernel_97257/3305748768.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(sql, con=raw)


,city_a,state_a,city_b,state_b,distance_km
0,sao paulo,SP,rio de janeiro,RJ,346.99
